In [1]:
import pandas as pd

# Load the uploaded dataset
file_path = '/content/cars.csv'
df = pd.read_csv(file_path)

# Show the first few rows and columns
df.head(), df.columns


(          Unnamed: 0   mpg  cyl   disp   hp  drat     wt   qsec  vs  am  gear  \
 0          Mazda RX4  21.0    6  160.0  110  3.90  2.620  16.46   0   1     4   
 1      Mazda RX4 Wag  21.0    6  160.0  110  3.90  2.875  17.02   0   1     4   
 2         Datsun 710  22.8    4  108.0   93  3.85  2.320  18.61   1   1     4   
 3     Hornet 4 Drive  21.4    6  258.0  110  3.08  3.215  19.44   1   0     3   
 4  Hornet Sportabout  18.7    8  360.0  175  3.15  3.440  17.02   0   0     3   
 
    carb  
 0     4  
 1     4  
 2     1  
 3     1  
 4     2  ,
 Index(['Unnamed: 0', 'mpg', 'cyl', 'disp', 'hp', 'drat', 'wt', 'qsec', 'vs',
        'am', 'gear', 'carb'],
       dtype='object'))

In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

# Load the dataset
df = pd.read_csv('cars.csv')  # or your full path if needed

# Define target and features
y = df['mpg'].values
X = df[['gear', 'cyl', 'carb', 'hp', 'disp', 'wt']]

# Specify categorical and numerical columns
cat_cols = ['gear', 'cyl', 'carb']
num_cols = ['hp', 'disp', 'wt']

# Setup preprocessing tools
encoder = OneHotEncoder(drop='first', sparse_output=False, handle_unknown='ignore')
scaler = StandardScaler()
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# Cross-validation loop
r2_scores = []
for train_idx, test_idx in kf.split(X):
    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test = y[train_idx], y[test_idx]

    # Fit encoder & scaler on training data only
    X_train_cat = encoder.fit_transform(X_train[cat_cols])
    X_train_num = scaler.fit_transform(X_train[num_cols])

    # Transform test data using fitted encoders
    X_test_cat = encoder.transform(X_test[cat_cols])
    X_test_num = scaler.transform(X_test[num_cols])

    # Combine features
    X_train_final = np.hstack((X_train_num, X_train_cat))
    X_test_final = np.hstack((X_test_num, X_test_cat))

    # Fit model and evaluate
    model = LinearRegression()
    model.fit(X_train_final, y_train)
    y_pred = model.predict(X_test_final)
    r2_scores.append(r2_score(y_test, y_pred))

# Output scores
print("R² scores per fold:", np.round(r2_scores, 3))
print("Average R²:", round(np.mean(r2_scores), 3))


/usr/local/lib/python3.11/dist-packages/sklearn/preprocessing/_encoders.py:246: UserWarning: Found unknown categories in columns [2] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


R² scores per fold: [0.656 0.226 0.623 0.199 0.779]
Average R²: 0.497
